# API (Application Programmable Interface)

Et API kan være så mangt, og det er derfor vanskelig å si noe generelt om APIer. Jeg vil fokusere på en gruppe web-baserte APIer som har følgende kjennetegn:

1. Et program genererer en spørring, som det sender til et web-API
2. Spørringen behandles av APIet
3. APIet returnerer informasjonen du ba om

Hvordan spørringene og responsene ser ut, og hvordan informasjonsoverføringen skjer, er bestemt av hvert enkelt API.

Vi skal gå igjennom et par enkle eksempler.

## Google Maps API for geokoding

Dette er et veldig enkelt API, som bare trenger en forholdsvis kort URL som inneholder en addresse, og returnerer en JSON streng med informasjon om addressen, inkludert koordinater.

Akkurat dette er en betalt tjeneste, og Google krever at vi inkluderer en unik nøkkel i forespørselen.

In [2]:
import requests
import json

In [3]:
key = '****'
address = 'Otervegen 23, Kongsvinger'

In [4]:
result = requests.get('https://maps.googleapis.com/maps/api/geocode/json?key=' + key + '&address=' + address).text
json_result = json.loads(result)

<pre>https://maps.googleapis.com/maps/api/geocode/json?key=***&address=Otervegen 23, Kongsvinger</pre>

Hele resultatet:

In [5]:
json_result

{'results': [{'address_components': [{'long_name': '23',
     'short_name': '23',
     'types': ['street_number']},
    {'long_name': 'Otervegen', 'short_name': 'Otervegen', 'types': ['route']},
    {'long_name': 'Kongsvinger',
     'short_name': 'Kongsvinger',
     'types': ['postal_town']},
    {'long_name': 'Kongsvinger',
     'short_name': 'Kongsvinger',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Hedmark',
     'short_name': 'Hedmark',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Norway',
     'short_name': 'NO',
     'types': ['country', 'political']},
    {'long_name': '2211', 'short_name': '2211', 'types': ['postal_code']}],
   'formatted_address': 'Otervegen 23, 2211 Kongsvinger, Norway',
   'geometry': {'location': {'lat': 60.17382749999999, 'lng': 12.0163764},
    'location_type': 'ROOFTOP',
    'viewport': {'northeast': {'lat': 60.1751764802915,
      'lng': 12.0177253802915},
     'southwest': {'lat': 6

Hvis vi bare trenger koordinatene, kan vi hente koordinat-elementene fra JSON-objektet:

In [6]:
lat = json_result['results'][0]['geometry']['location']['lat']
lng = json_result['results'][0]['geometry']['location']['lng']
print(lat, lng)

60.17382749999999 12.0163764


## Statistikkbanken

Dette er gammelt nytt for mange, men statistikkbanken har et API som mange, inkludert oss selv, bruker. Kristian Lønø har holdt kurs i å bruke APIet fra SAS, men her er et eksempel i Python, hentet rett fra ssb.no. 

I eksempelet over ble addressen vi ville ha koordinater for som en del av URLen. Her vil ikke spørringen vår være synlig i selve URLen, men sendes på samme måte som data sendes når du fyller inn et skjema på nett.

In [7]:
EXAMPLE_URL = 'http://data.ssb.no/api/v0/no/table/03024'

In [8]:
payload = {
    "query": [
        {"code": "VareGrupper2", 
         "selection": {
             "filter": "item", 
             "values": ["01", "02"] 
         } 
        }, 
        {
            "code": "ContentsCode", 
            "selection": {
                "filter": "item", 
                "values": ["Vekt", "Kilopris"] 
            } 
        }, 
        {
            "code": "Tid", 
            "selection": {
                "filter": "top", 
                "values": ["10"] 
            } 
        } 
    ], 
    "response": {"format": "csv"} 
}

In [9]:
data = requests.post(EXAMPLE_URL, json = payload)

In [10]:
data.text

'"varegruppe","Vekt (tonn) 2017U34","Vekt (tonn) 2017U35","Vekt (tonn) 2017U36","Vekt (tonn) 2017U37","Vekt (tonn) 2017U38","Vekt (tonn) 2017U39","Vekt (tonn) 2017U40","Vekt (tonn) 2017U41","Vekt (tonn) 2017U42","Vekt (tonn) 2017U43","Kilopris (kr) 2017U34","Kilopris (kr) 2017U35","Kilopris (kr) 2017U36","Kilopris (kr) 2017U37","Kilopris (kr) 2017U38","Kilopris (kr) 2017U39","Kilopris (kr) 2017U40","Kilopris (kr) 2017U41","Kilopris (kr) 2017U42","Kilopris (kr) 2017U43"\r\n"Fersk oppalen laks",16988,18043,18273,19307,18876,19317,18964,18213,18824,20343,56.80,54.89,55.56,53.85,55.30,57.21,56.61,54.52,55.32,53.17\r\n"Frosen oppalen laks",435,366,374,285,553,584,472,569,511,506,62.85,61.28,58.39,59.66,59.15,59.18,59.77,59.71,59.82,61.92\r\n'

## Twitter

Twitter har et eget API, som f.eks. lar deg søke i tweets (med en del begrensninger). Som med google krever dette APIet at du er registrert som bruker, men twitter krever langt mer omfattende autentisering enn Google. Vi bruker derfor et eget bibliotek for å håndtere autentiseringen.

In [ ]:
import oauth2

In [ ]:
CONSUMER_KEY = '***'
CONSUMER_SECRET = '***'
ACCESS_KEY = '***'
ACCESS_SECRET = '***'

In [ ]:
consumer = oauth2.Consumer(key=CONSUMER_KEY, secret=CONSUMER_SECRET)
token = oauth2.Token(key=ACCESS_KEY, secret=ACCESS_SECRET)
client = oauth2.Client(consumer, token)

In [ ]:
query = '%23trump'

In [ ]:
url = 'https://api.twitter.com/1.1/search/tweets.json?q=' + query + '&count=5&response_type=recent'
response = client.request(url, method = 'GET')

In [ ]:
json_r = json.loads(response[1])
print(json_r['statuses'][0]['text'])

In [ ]:
for status in json_r['statuses']:
    print(status['text'], '\n')

In [ ]:
print(json_r['statuses'][3])

## Twitter streaming med tweepy-biblioteket

Importere nødvendig kode

In [11]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import Stream

In [12]:
CONSUMER_KEY = '***'
CONSUMER_SECRET = '***'
ACCESS_KEY = '***'
ACCESS_SECRET = '***'

Autentisering med innebygget OAuth handler.

In [13]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

Må først lage en liten funksjon som definerer hva som skal skje når vi mottar en ny tweet, eller en feilmelding

In [16]:
class StdOutListener(StreamListener):
    """ A listener handles tweets that are received from the stream.
    This is a basic listener that just prints received tweets to stdout.
    """
    
    def on_data(self, data):
        print(json.loads(data), '\n')
        return True

    def on_error(self, status):
        print(status)

In [17]:
l = StdOutListener()
stream = Stream(auth, l)
stream.filter(track=['Statistics', 'Statistikk'])

{'retweeted': False, 'in_reply_to_status_id_str': None, 'lang': 'ja', 'user': {'description': '人文科学系の洋書や英字新聞（Japan Times）に使われている、TOEIC750（英検準一級レベル）以上の単語やフレーズを主にツイートします。\u3000※ミスがあればお知らせください。', 'following': None, 'followers_count': 6264, 'url': None, 'name': 'ハイレベル英単語', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'verified': False, 'id_str': '1227867133', 'profile_sidebar_border_color': 'C0DEED', 'time_zone': None, 'follow_request_sent': None, 'is_translator': False, 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/3736424842/13bfd3d317c1220acc136b0da28f1078_normal.jpeg', 'lang': 'ja', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_text_color': '333333', 'contributors_enabled': False, 'location': None, 'listed_count': 150, 'friends_count': 2, 'profile_use_background_image': True, 'translator_type': 'none', 'profile_link_color': '1DA1F2', 'statuses_count': 78690, 'created_at': 'Th

KeyboardInterrupt: 